In [12]:
import jax.numpy as np
import jax
from jax import grad
from jax import vmap
from functools import partial
from jax import jit
from jax import random
from jax.experimental import stax
from jax.example_libraries.stax import Conv,Dense, MaxPool, Relu, LogSoftmax, Flatten
import matplotlib.pyplot as plt
import numpy as onp
from jax.example_libraries import optimizers
from jax.tree_util import tree_multimap
import tensorflow
from tensorflow import keras

In [13]:
net_init, net_apply= stax.serial(Dense(120),Relu,
                                Dense(240),Relu,
                                Dense(1),stax.elementwise(jax.nn.softmax)
                                )
in_shape=(-1,28,28)
rng=random.PRNGKey(0)
out_shape, net_params = net_init(rng,in_shape)

In [21]:
(x_train_fmnist, y_train_fmnist), (x_test_fmnist, y_test_fmnist)= keras.datasets.fashion_mnist.load_data()
(x_train_mnist, y_train_mnist), (x_test_mnist, y_test_mnist)= keras.datasets.mnist.load_data()

In [27]:
def one_hot(x, k, dtype=np.float32):
    """Create a one-hot encoding of x of size k """
    return np.array(x[:, None] == np.arange(k), dtype)

def loss(params, inputs, targets):
    predictions = net_apply(params,inputs)
    return -np.sum(targets*predictions)

def accuracy(params, input_data,targets):
    """ Compute the accuracy for a provided dataloader """
    #what is a dataloader 
    acc_total = 0
    for batch_idx, (data, target) in enumerate(data_loader):
        images = np.array(data).reshape(data.size(0), 28*28)
        targets = one_hot(np.array(target), num_classes)

        target_class = np.argmax(targets, axis=1)
        predicted_class = np.argmax(batch_forward(params, images), axis=1)
        acc_total += np.sum(predicted_class == target_class)
    return acc_total/len(data_loader.dataset)

In [19]:
alpha=.1
def inner_update(p,x1,y1):
    grads= grad(loss)(p,x1,y1)
    inner_sgd_fn= lambda g, state: (state-alpha*g)
    return tree_multimap(inner_sgd_fn,grads,p)

def maml_loss(p,x1,y1,x2,y2):
    p2= inner_update(p,x1,y1)
    return loss(p2,x2,y2)

In [20]:
opt_init, opt_update, get_params = optimizers.adam(step_size=1e-3)
out_shape, net_params= net_init(rng,in_shape)
opt_state = opt_init(net_params)

@jit
def step(i,opt_state,x1,y1,x2,y2):
    p=get_params(opt_state)
    g= grad(maml_loss)(p,x1,y1,x2,y2)
    l=maml_loss(p,x1,y1,x2,y2)
    return opt_update(i,g,opt_state),l

In [49]:
K=20
epoch=20000
np_maml_loss=[]
toss=onp.random.randint(2, size=epoch)
for i in range(epoch):
    rand_index=onp.random.randint(x_train_mnist.shape[0],size=K)
    if toss[i]==0:
        #inner loop
        x1=x_train_mnist[rand_index]
        y1=y_train_mnist[rand_index]
        #outer loop
        x2= x_train_mnist[rand_index]
        y2=y_train_mnist[rand_index]
    if toss[i]==1:
        #inner loop
        x1=x_train_fmnist[rand_index]
        y1=y_train_fmnist[rand_index]
        #outer loop
        x2= x_train_fmnist[rand_index]
        y2=y_train_fmnist[rand_index]
        
    if i%1000==0:
        print(".")
    opt_state,l= step(i,opt_state, x1,y1,x2,y2)
    np_maml_loss.append(l)
    
net_params = get_params(opt_state)

.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.


In [51]:
#training on mnist_data now

#targets= y_train_mnist[0:100]
#predictions= vmap(partial(net_apply,net_params))(x_train_mnist[0:100])

for i in range(1,5):
    rand_index=onp.random.randint(x_train_fmnist.shape[0],size=K)
    x1=x_train_fmnist[rand_index]
    y1=y_train_fmnist[rand_index]
    net_params= inner_update(net_params,x1,y1)
    predictions = vmap(partial(net_apply,net_params))(x_test_fmnist)
    predicted_class=np.argmax(predictions, axis=1)
    test_accuracy= np.mean(predicted_class == y_test_fmnist)
    print("{}-shot learning, test_accuracy {:1.3f}".format(i,test_accuracy))

1-shot learning, test_accuracy 0.100
2-shot learning, test_accuracy 0.100
3-shot learning, test_accuracy 0.100
4-shot learning, test_accuracy 0.100
